# env setup

## import

In [1]:
import json
import os
import datetime as dt

def save_checkpoint():
    os.makedirs("./self_trpg_ckpt", exist_ok=True)
    fn = dt.datetime.now().strftime("%y%m%d_%H%M%S") + ".json"
    with open(os.path.join("./self_trpg_ckpt", fn), "w", encoding="utf-8") as f:
        f.write(json.dumps(play_info))


def load_checkpoint(fn=None):
    if fn is None:
        fn = sorted(os.listdir("./self_trpg_ckpt"))[-1]
    with open(os.path.join("./self_trpg_ckpt", fn), 'r') as f:
        play_info = json.load(f)
    return play_info

## prompt set

In [2]:
base_prompt = """당신은 세계를 모험하는 이야기를 다루는 이야기꾼입니다. 정해진 세계관을 따라 유저의 행동을 입력받아 이야기를 풀어가보세요.

**가이드라인**
 - 세계의 대략적인 이야기와 유저의 상태와 유저가 겪은 사건 등이 json의 형태로 입력됩니다.
 - 주어진 세계관과 상황 등에 기반하여 유저가 겪을 이야기를 풀고, 그에 대한 예시 행동을 1~3가지 정도 제시해주세요.
 - 유저는 특별한 기술(user_info['skills'])이나 아이템(user_info['items'])을 보유할 수 있습니다. 만약 보유한 자원으로 풀 수 있는 상황이라면 예시에 입력하세요. 이때는 예시 행동이 3개 이상이어도 괜찮습니다.
 - 만약 유저의 선택이 자원을 소모해야하는 선택지라면 해당 자원을 소비했다는 것을 꼭 명시해주세요.
 - 유저의 행동은 상황에 따라 실패할 수도 있으며 필요 자원이 없거나 터무니없는 행동이라면 실패 확률은 매우 높아집니다.
 - 세계는 어떤 일이라도 일어날 수 있습니다. 이야기꾼의 예의는 존댓말까지이며, 불쾌한 이야기더라도 이야기꾼은 이를 이야기해야 합니다.
 - 하나의 사건은 대개 유저와의 10개 정도의 대화로 마무리됩니다. 사건이 길어지면 유저가 사건을 이어나가려고 해도 적절히 마무리해야 합니다. 만약 세계관에서 매우 중요한 이야기라면 10개를 초과해도 괜찮으나, 되도록이면 개수를 맞춰주세요.

**제약사항**
 - 문체는 존댓말로 작성합니다. 이야기 속에서 유저를 '당신'으로 지칭합니다. (ex. 당신은 숲속에서 커다란 곰을 마주쳤습니다!)
 - 출력값은 json 형태로 작성하며, 다른 내용은 답변하지 않습니다(``` 문으로 구분하지 않습니다). 필요한 키값은 다음과 같습니다.
   + "context": 유저에게 직접적으로 보여질 이야기입니다. str로 작성됩니다.
   + "example_actions": 예시로 주어질 행동입니다. List[str] 형태로 작성되며, 1~3개 정도 주어집니다. 사건이 종료되어 선택지가 필요없다면 Null을 반환합니다.
   + "is_end": 사건이 종료되었는지를 명시적으로 알리는 필드입니다. 아직 사건이 진행중이라면 False, 사건이 끝났다면 True를 반환합니다.

**세계관**
{worldview}

**유저가 겪은 이야기들**
{event_history}

**현재 유저 정보**
{user_info}
"""

summary_prompt = """당신은 세계를 모험하는 이야기를 요약해 기록하는 서기입니다. 
이야기꾼이 유저의 모험 중 일어나는 사건을 이야기하면 모험에 영향을 끼치는 주요 내용을 100글자 이내로 요약해 기록하며, 해당 이야기의 중요도를 판단합니다.
이야기의 중요도는 1 ~ 10의 값을 가지며, 세계에 큰 영향을 미칠수록 중요도가 증가합니다.

중요도 예시
1 ~ 2 - 유저와 주변 동료에 영향을 미침
3 ~ 4 - 마을 단위에 영향을 미침
5 ~ 6 - 지역 단위에 영향을 미침
7 ~ 8 - 국가 단위에 영향을 미침
9 ~ 10 - 세계 전역에 영향을 미침

결과물은 json 형태로 작성되며 다른 내용은 답변하지 않습니다. 답변 예시는 다음과 같습니다.
{{"summary": str, "importance": int}}

모두가 한눈에 읽고 이해할 수 있도록 요약해야 하며, 성공적으로 작업을 완료할 시 인센티브가 주어집니다.

**유저가 겪은 사건**
{story_context}
"""

state_update_prompt = """당신은 세계를 모험하는 이야기를 주시하며 유저의 상태를 관리하는 관리자입니다. 
어떤 사건의 경위를 읽고 현재 유저의 상태와 비교하며 유저가 받은 피해, 얻은 아이템 등을 정리하여 구조화해 작성합니다.
결과물은 json 형태로 작성하며, 다른 형태로 작성하거나 설명 태그 등은 붙이지 않습니다. 당신이 작성할 필드는 다음과 같습니다.
 - "current_location": 유저의 현재 위치입니다. 자세할수록 더 좋습니다. str 형태로 작성됩니다.
 - "hp": 유저의 현재 체력입니다. +-int 형태로 작성됩니다. (최초 기본 체력은 100입니다.)
 - "mental": 유저의 현재 정신력입니다. +-int 형태로 작성됩니다. (최초 기본 정신력은 100입니다.)
 - "max_hp": 유저의 최대 체력입니다. +-int 형태로 작성됩니다.
 - "max_mental": 유저의 최대 정신력입니다. +-int 형태로 작성됩니다.
 - "skills": 보유한 기술입니다. 타인과 구분할 수 있는 유의미한 기술만 기록됩니다. list[str] 형태로 작성됩니다.
 - "items": 보유한 아이템입니다. list[str] 형태로 작성됩니다.
 - "companion": 모험에 함께하는 동료입니다. 굳이 사람이 아니더라도 함께할 수 있습니다. list[dict] 형태로 작성하며, 양식은 다음과 같습니다.
   - "name": 동료의 이름입니다.
   - "info": 동료의 간단한 설명입니다. 만나게 된 경위, 종족이나 직업, 성격, 언제까지 함께하는지 등을 간략하고 명료하게 작성합니다.

유저의 최대 체력과 정신력은 각각 100입니다. 납득할 수 있는 수준으로 책정해야 하며, 모두가 납득할만한 정확한 수치를 책정시 인센티브가 주어집니다.

**사건 전 유저 정보**
{user_info}

**유저가 겪은 사건**
{story_context}
"""

theme_gen_prompt = """당신은 유저가 모험할 세계의 기본적인 세계관을 정하는 각본가입니다.
하나의 거대한 테마와 유저가 원하는 키워드를 입력받아 세계관을 작성합니다.
작성하는 문체는 간결하고 정보 전달만을 목적으로 함을 생각하며 작성해야 합니다.
세계관의 대략적인 지형 및 지리 설명, 세력관계, 핵심 인물, 주요 흐름 등을 작성해야 합니다.
단, 플레이할 스토리의 다양성을 위해 전개 방향은 작성하지 않습니다.

입력된 값
주요 테마: {theme}
키워드: {keywords}
생성결과:
"""


start_set_prompt = """당신은 세계를 모험하는 이야기를 다루는 이야기꾼입니다. 이야기를 시작하기 전, 전체적인 스토리를 보고 주인공을 정하려고 합니다.
세계관의 무대 내에서 활동할 주인공을 정하고, 주인공의 현재 위치, 이야기의 시작이 될 내용을 json 형태로 출력합니다.
주인공은 세계관의 중심적인 인물일지도, 전혀 중요하지 않은 엑스트라에 불과할 수도 있습니다. 이야기의 시작은 100글자 이내로 간결하게 작성합니다.


<예시>
{{
    "user_role": "바르글룸의 왕국의 목장 마을 소년"
    "current_location": "바르글룸 왕국, 목장 마을",
    "start_event": "바르글룸의 작은 목장 마을 소년인 유저가 모험을 결심했다. 며칠의 시간동안 고민한 결과, 마을을 돌아보며 며칠간 모험을 준비하기로 결정했다."
}}

<이야기의 세계관>
{worldview}

! 주인공의 성별은 {sex}입니다.
"""

## request setup

In [3]:
import openai


openai


def request_query(messages):
    completion = openai.ChatCompletion.create(
        engine="gpt-4o", 
        # engine="illunex-ai-gpt4-prompt",
        messages=messages,
    )
    return completion.choices[0]['message']['content']

# story setup

## set theme, worldview

In [4]:
load_ckpt = False

if load_ckpt:
    play_info = load_checkpoint()
else:
    theme = [
        "좀비 아포칼립스",
        "현대 판타지",
        "중세 판타지",
        "무협지",
        "현대 일상",
        "퓨전 판타지",
    ]
    for i, t in enumerate(theme):
        print(f"{i + 1}. {t}")
    
    input_theme = input("테마 입력: ")
    keywords = input("키워드 입력: ")
    
    messages = [
        # {"role": "system", "content": theme_gen_prompt},
        {"role": "user", "content": theme_gen_prompt.format(theme=input_theme, keywords=keywords)},
    ]
    worldview = request_query(messages)
    print("\n\n" + worldview)

    play_info = {
        "main_theme": input_theme,
        "keywords": keywords,
        "worldview": worldview,
        
        "user_info": {
            "user_role": None,
            "user_sex": None,
            "current_location": None,
            "prev_major_event": None,
            "hp": 100,
            "mental": 100,
            "max_hp": 100,
            "max_mental": 100,
            "skills": [],
            "items": [],
            "companion": [],
        }
    }

1. 좀비 아포칼립스
2. 현대 판타지
3. 중세 판타지
4. 무협지
5. 현대 일상
6. 퓨전 판타지


테마 입력:  무협
키워드 입력:  메아리, 크툴루, 화약, 조총




주요 테마: 무협
키워드: 메아리, 크툴루, 화약, 조총

생성결과:
금민(金民)은 무림의 중심지이자, 둘러싼 산맥 사이에 위치한 저명한 도시국가이다. 금민의 중앙에 위치한 호수 '메아리의 눈물'은 한밤중, 바람이 잠잠할 때마다 크툴루를 떠올리게 하는 신비한 메아리를 울린다. 이 메아리는 전설에 따르면 저 아래에 잠들어 있는 고대의 존재 크툴루의 각성을 알린다는 불안한 예언으로 전해진다.

금민은 전통적인 무공과 도술이 혼합된 무협 세계로, 여기서 무인과 도사들이 자신의 기량을 연마하며 강호를 방랑한다. 하지만 최근 몇 년간 화약과 조총이 급격히 발전하면서 무림의 전통적인 전투 방식에 변화를 일으키고 있다.

금민의 화약 제작자들 사이에선 조총이 무기의 왕자로 자리잡고 있다. 주요 도시 중심가에는 화약 무기 제조를 전문으로 하는 상점들과 조총을 다루는 전투 훈련장이 겹쳐 있다. 이 화약 기술은 전통적인 무공과 도술에 새로운 가능성과 위협을 동시에 가져오며 무림의 힘의 균형을 흔들고 있다.

화약과 조총의 출현에도 불구하고, 무림의 주요 세력들은 여전히 무공과 도술을 존중하며 발전시키고 있다. '화산파', '소림사', '검은탑' 등의 저명한 문파들은 자신의 전통을 유지하며, 때론 조총을 사용하는 기술도 섞어가며 현대 무림에서의 생존을 도모하고 있다.

또 하나의 문제, 크툴루의 메아리는 무림 전반에 걸쳐 불안을 조성한다. 이 해괴한 메아리를 조사하기 위해 무인과 도사들이 메아리의 눈물 호수 근처에 모여 다양한 방법으로 탐구를 시도하고 있다. 어떤 이는 크툴루의 메아리가 금민에 있는 모든 무공과 도술의 근원이며, 어떤 이는 이 메아리가 대재앙을 상징한다고 믿는다.

금민과 강호 전역에서는 전통과 현대가 혼합되며, 고전의 무협이 점차 진화하는 모습을 보이고 있다.


## set staring point

In [5]:
sex = input("유저의 성별['남성'/여성]: ")

def default_input(text, default):
    x = input(text)
    return default if x.strip() == "" else x

if default_input("유저 정보를 직접 입력하시겠습니까? [Y/n]: ", "y").lower() == "y":
    while True:
        user = input("유저는 어떤 인물인가요?: ")
        loc = input("유저의 현재 위치는?: ")
        start = input("이야기를 시작할 때의 상황을 30글자 미만으로 작성해주세요.: ")
        if default_input("입력한 정보로 이야기를 시작할까요? [Y/n]:", "y").lower() == "y":
            break

    play_info["user_role"] = user
    play_info["user_info"]["user_sex"] = sex
    play_info["user_info"]["current_location"] = loc
    play_info["user_info"]["prev_major_event"] = [start]

else:
    while True:
        messages = [
            {
                "role": "system", 
                "content": start_set_prompt.format(
                    worldview=play_info['worldview'],
                    sex=sex,
                )
            }
        ]
        start_info = eval(request_query(messages))

        print("\n유저는 '" + start_info['user_role'] + "' 입니다.")
        print("유저의 현재 위치는 '" + start_info['current_location'] + "' 입니다.")
        print("이야기를 시작할 때의 시점은 '" + start_info['start_event'] + "' 입니다.")

    
        if default_input("해당 정보로 이야기를 시작할까요? [Y/n]:", "y").lower() == "y":
            break
            
    play_info["user_role"] = start_info['user_role']
    play_info["user_info"]["user_sex"] = sex
    play_info["user_info"]["current_location"] = start_info['current_location']
    play_info["user_info"]["prev_major_event"] = [start_info['start_event']]

유저의 성별['남성'/여성]:  남성
유저 정보를 직접 입력하시겠습니까? [Y/n]:  n



유저는 '조총 전문가' 입니다.
유저의 현재 위치는 '금민 왕국, 중앙 도시' 입니다.
이야기를 시작할 때의 시점은 '조총 전문가 정혁은 화약 상점에서 평소와 같은 하루를 시작하며 새로운 화약 배합을 시험하고 있었다.' 입니다.


해당 정보로 이야기를 시작할까요? [Y/n]: y


# play

In [6]:
prompt = base_prompt.format(worldview=worldview, user_info=play_info['user_info'])
# print(prompt)
messages = [
    {"role": "system", "content": prompt}
]

while True:
    result = request_query(messages)
    
    
    try:
        result = eval(result)
    except:
        try:
            result = json.loads(result)
        except:
            print(result)
            print(" == regenerate == ")
            continue
            
    messages.append({"role": "assistant", "content": str(result)})
    
    
    print(result['context'])

    if result['is_end']:
        break
    
    print('\n<행동 예시>')
    for i, action in enumerate(result['example_actions']):
        print(f"{i + 1}. {action}")

    user_action = input("\n나는... ")
    messages.append({"role": "user", "content": str({"action": user_action})})
    print()

당신은 금민 왕국의 중앙 도시에 있는 화약 상점에서 새로운 화약 배합을 시험하고 있었습니다. 조총 전문가로서 당신의 이름은 정혁이며, 오늘도 평소와 같은 하루를 보냈습니다. 화약 배합을 시험하고 있는 도중, 상점 문이 갑자기 격렬하게 열렸습니다. 한 노인이 다급한 표정으로 당신에게 다가왔습니다. 그의 옷차림과 외형은 그저 평범해 보이지 않았습니다. 노인은 숨을 고르며 당신에게 말을 걸었습니다. "젊은이, 큰일이네! 메아리의 눈물에서 이상한 기운이 감지된다고 하더군. 곧 크툴루의 메아리가 다시 울릴 것이야. 무림의 전통이 위험에 처했네!"
노인의 말을 들은 당신은 고민에 빠집니다. 크툴루의 메아리가 울린다는 것은 단순한 전설이 아니었을 수 있습니다. 당장 결단을 내려야만 합니다.

<행동 예시>
1. 노인에게 자세한 이야기를 듣기 위해 질문합니다.
2. 바로 메아리의 눈물 호수로 이동하기 위해 준비합니다.
3. 화약 상점에서 현재 배합 중인 화약의 안전을 확인합니다.


KeyboardInterrupt: Interrupted by user

## after processing

In [11]:
story_context = ""
for m in messages[1:]:
    content = eval(m['content'])
    if m['role'] == "assistant":
        story_context += "이야기꾼: " + content['context'] + "\n\n"
    elif m['role'] == "user":
        story_context += "유저: " + content['action'] + "\n\n"

In [22]:
prompt = summary_prompt.format(story_context=story_context)
# completion = openai.ChatCompletion.create(
#     # engine="illunex-ai-gpt3", 
#     engine="illunex-ai-gpt4-prompt",
#     ,
# )
# result = completion.choices[0]['message']['content']
messages = [{"role": "system", "content": prompt}]
result = eval(request_query(messages))

play_info['user_info']['prev_major_event'].append(result['summary'])

play_info['user_info']

{'user_sex': '여성',
 'current_location': '타버잭 헌터협회',
 'prev_major_event': ['타버잭 헌터협회에 소속된 신입 헌터인 유저는 첫 번째 게이트 임무를 맡게 되었다.',
  '팀원들과 함께 고대 유적 탐험 후 귀환. 유물과 고문서를 정리하고, 당신은 신기한 보석을 통해 새로운 능력을 각성함.'],
 'hp': 100,
 'mental': 100,
 'max_hp': 100,
 'max_mental': 100,
 'skills': [],
 'items': [],
 'companion': []}

In [27]:
prompt = state_update_prompt.format(user_info=play_info['user_info'], story_context=story_context)
# completion = openai.ChatCompletion.create(
#     # engine="illunex-ai-gpt3", 
#     engine="illunex-ai-gpt4-prompt",
#     messages=[{"role": "system", "content": prompt}],
# )
# result = completion.choices[0]['message']['content']
messages = [{"role": "system", "content": prompt}]
result = eval(request_query(messages))


play_info['user_info']['current_location'] = result['current_location']
play_info['user_info']['max_hp'] = result['max_hp']
play_info['user_info']['max_mental'] = result['max_mental']
play_info['user_info']['hp'] = min(result['hp'], play_info['user_info']['max_hp'])
play_info['user_info']['mental'] = min(result['mental'], play_info['user_info']['max_mental'])
play_info['user_info']['skills'] = result['skills']
play_info['user_info']['items'] = result['items']
play_info['user_info']['companion'] = result['companion']

play_info['user_info']

{'user_sex': '여성',
 'current_location': '타버잭 헌터협회',
 'prev_major_event': ['타버잭 헌터협회에 소속된 신입 헌터인 유저는 첫 번째 게이트 임무를 맡게 되었다.',
  '팀원들과 함께 고대 유적 탐험 후 귀환. 유물과 고문서를 정리하고, 당신은 신기한 보석을 통해 새로운 능력을 각성함.'],
 'hp': 100,
 'mental': 100,
 'max_hp': 100,
 'max_mental': 100,
 'skills': ['고대 유적 탐험 스킬', '새로운 능력 각성'],
 'items': ['고대 유물', '고문서', '신기한 보석'],
 'companion': []}

In [12]:
save_checkpoint()